In [1]:
import numpy as np
import matplotlib.pyplot as plt
import shutil
import pandas as pd
import cv2
import os
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from tensorflow.keras.layers import Dense, BatchNormalization
import tensorflow as tf
print(keras.__version__)
print(tf.__version__)

2.5.0
2.5.0


In [ ]:
'''metadata=pd.read_csv("metadata.csv")

In [ ]:
'''metadata.head(10)

In [ ]:
'''IMAGES_PATH="kaggle/images"
target_dir="datasets/datasets/train/positive"
for (i,row) in metadata.iterrows():
    if row["finding"]=="COVID-19" and row["view"]=="PA":
        filename=row["filename"]
        image_path=os.path.join(IMAGES_PATH,filename)
        image_copy_path=os.path.join(target_dir,filename)
        #print(image_path)
        shutil.copy2(image_path,image_copy_path)
''''        

In [2]:
idg=tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20, # You can uncomment these parameters to make you generator rotate & flip the images to put the train model in stricter conditions.
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    vertical_flip=False,
    validation_split=0.2
)

In [3]:
train_gen = idg.flow_from_directory("data1",
                                                   target_size=(128,128),                                                   
                                                   subset='training',
                                                   class_mode='binary',
                                                   batch_size=32,
                                                   shuffle=True,
                                                   seed=1
                                                )

Found 18126 images belonging to 2 classes.


In [4]:
val_gen = idg.flow_from_directory("data1",
                                                   target_size=(128,128),                                                   
                                                   subset='validation',
                                                   class_mode='binary',
                                                   batch_size=32,
                                                   shuffle=True,
                                                   seed=1
                                                )

Found 4531 images belonging to 2 classes.


In [5]:
train_gen.class_indices

{'covid': 0, 'normal': 1}

In [6]:
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

In [7]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
input_shape = (128,128, 3)
googleNet_model =ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
googleNet_model.trainable = True

model = Sequential()
model.add(googleNet_model)
model.add(GlobalAveragePooling2D())

model.add(Dense(units=2, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 4, 4, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 23,591,810
Trainable params: 23,538,690
Non-trainable params: 53,120
_________________________________________________________________


C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [8]:
checkpoint=tf.keras.callbacks.ModelCheckpoint(
   '2resnet50.h5',
    save_best_only=True)




EPOCHS = 20
BATCH_SIZE =32
history = model.fit(train_gen, batch_size = BATCH_SIZE, epochs = EPOCHS, validation_data = val_gen, verbose = 1,callbacks=checkpoint)

Epoch 1/20
567/567 [==============================] - 213s 353ms/step - loss: 0.3754 - accuracy: 0.8263 - val_loss: 1.2766 - val_accuracy: 0.4423


C:\Users\user\anaconda3\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


Epoch 2/20
567/567 [==============================] - 140s 247ms/step - loss: 0.2190 - accuracy: 0.9094 - val_loss: 0.4745 - val_accuracy: 0.8029
Epoch 3/20
567/567 [==============================] - 148s 261ms/step - loss: 0.1716 - accuracy: 0.9313 - val_loss: 0.2662 - val_accuracy: 0.8947
Epoch 4/20
567/567 [==============================] - 143s 252ms/step - loss: 0.1418 - accuracy: 0.9421 - val_loss: 0.2712 - val_accuracy: 0.8949
Epoch 5/20
567/567 [==============================] - 150s 265ms/step - loss: 0.1226 - accuracy: 0.9496 - val_loss: 0.1198 - val_accuracy: 0.9574
Epoch 6/20
567/567 [==============================] - 140s 247ms/step - loss: 0.1106 - accuracy: 0.9577 - val_loss: 0.1789 - val_accuracy: 0.9369
Epoch 7/20
567/567 [==============================] - 138s 243ms/step - loss: 0.0998 - accuracy: 0.9617 - val_loss: 0.1121 - val_accuracy: 0.9623
Epoch 8/20
567/567 [==============================] - 134s 236ms/step - loss: 0.0902 - accuracy: 0.9659 - val_loss: 0.1128 -

In [10]:
test_datagen = image.ImageDataGenerator(rescale=1./255)
test_gen = test_datagen.flow_from_directory(
        'covid',
        target_size=(128,128),
        batch_size=1,
        class_mode='binary',
        shuffle=False)
pred= model.predict(test_gen, verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)
count0=0
count1=0
for val in range(len(pred)):
    if predicted_class_indices[val]==0:
        count0=count0+1
    else:
        count1=count1+1
print("all_positive")        
print(count0)
print(count1)
print(" ")

############################################

test_gen = test_datagen.flow_from_directory(
        'data/cov',
        target_size=(128,128),
        batch_size=1,
        class_mode='binary',
        shuffle=False)
pred= model.predict(test_gen, verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)
count0=0
count1=0
for val in range(len(pred)):
    if predicted_class_indices[val]==0:
        count0=count0+1
    else:
        count1=count1+1
print("all_positive")        
print(count0)
print(count1)
print(" ")

##############################################

test_gen = test_datagen.flow_from_directory(
        'data/nor',
        target_size=(128,128),
        batch_size=1,
        class_mode='binary',
        shuffle=False)
pred= model.predict(test_gen, verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)
count0=0
count1=0
for val in range(len(pred)):
    if predicted_class_indices[val]==0:
        count0=count0+1
    else:
        count1=count1+1
print("all_negative")
print(count0)
print(count1)
print(" ")

###############################################
test_gen = test_datagen.flow_from_directory(
        'norm',
        target_size=(128,128),
        batch_size=1,
        class_mode='binary',
        shuffle=False)
pred= model.predict(test_gen, verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)
count0=0
count1=0
for val in range(len(pred)):
    if predicted_class_indices[val]==0:
        count0=count0+1
    else:
        count1=count1+1
print("all_negative")
print(count0)
print(count1)
print(" ")




#data=[]
#path='test/DLAI3_CXR_Validation_Set'
#for img in os.listdir(path): 
#    data.append(img) 

#df=pd.DataFrame(data,columns = ['Filename'])
#df['label']=0
#df['label']=predicted_class_indices
#df['label'].replace({1:0, 0:1}, inplace=True) 
#df.to_csv('InceptionResNetV2.csv',index=False)

Found 218 images belonging to 1 classes.
218/218 [==============================] - 8s 37ms/step
all_positive
193
25
 
Found 3616 images belonging to 1 classes.
3616/3616 [==============================] - 79s 22ms/step
all_positive
3368
248
 
Found 8000 images belonging to 1 classes.
8000/8000 [==============================] - 180s 23ms/step
all_negative
508
7492
 
Found 2192 images belonging to 1 classes.
2192/2192 [==============================] - 50s 23ms/step
all_negative
157
2035
 


In [ ]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[224,224, 3]))

cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.25))

cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.25))

cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Dropout(0.25))


cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=64, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [ ]:
cnn.summary()

In [ ]:
checkpoint=tf.keras.callbacks.ModelCheckpoint( '7.h5',save_best_only=True)
cnn.fit(x = train_gen, validation_data = val_gen, epochs =20,callbacks=checkpoint )

In [22]:
test_gen = test_datagen.flow_from_directory(
        'data/nor',
        target_size=(128,128),
        batch_size=1,
        class_mode='binary',
        shuffle=False)
pred= model.predict(test_gen, verbose=1)
predicted_class_indices=np.argmax(pred,axis=1)
count0=0
count1=0
for val in range(len(pred)):
    if predicted_class_indices[val]==0:
        count0=count0+1
    else:
        count1=count1+1
print("test")
print(count0)
print(count1)

Found 8000 images belonging to 1 classes.
8000/8000 [==============================] - 94s 12ms/step
test
822
7178


In [ ]:
from keras.models import load_model
model=load_model("2resnet50.h5")

In [ ]:
resnet50=94.07
inceptionv3=84.32